In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [5]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

In [ ]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source='user')
message

TextMessage(id='cc3fb894-3e6b-40e6-abb1-97f7f479e460', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 5, 10, 48, 52, 287198, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

In [7]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

In [11]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Sure thing! Just pack your umbrella and your sense of humor—trust me, you'll need both!"

In [12]:
import os
import sqlite3

db_name = 'tickets.db'
if os.path.exists(db_name):
    os.remove(db_name)

conn = sqlite3.connect(db_name)
c = conn.cursor()
c.execute('CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)')
conn.commit()
conn.close()

In [15]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [16]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [17]:
get_city_price("Rome")

499.0

In [18]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [19]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_aRlepPlyN1iMQQ848yCEUxZi', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_aRlepPlyN1iMQQ848yCEUxZi', is_error=False)]


"A roundtrip ticket to London will set you back $299. That's cheaper than a therapy session for all the jet lag you'll have! 🛫💼"